In [22]:
#mobilenetv3
#training process skip if you have the weights and run the below cell

import torch
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to tensors
])

# Load dataset
train_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\processed resized data\augmented ali data\balanced_dataset\train", transform=transform)
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\processed resized data\augmented ali data\balanced_dataset\valid", transform=transform)

# Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))  # Use 80% of the data for training
# val_size = len(dataset) - train_size  # Use the remaining 20% for validation
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_size, batch_size=128, shuffle=True)
val_loader = DataLoader(val_size, batch_size=128, shuffle=False)

# Load pre-trained MobileNetV3 model
model = torchvision.models.mobilenet_v3_small(pretrained=True)

# Modify the model's top layers for your specific task
num_classes = len(train_size.classes)  # Number of classes in your dataset
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, num_classes)

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 50

# Initialize early stopping variables
best_val_loss = float('inf')
patience = 7  # Number of epochs to wait for validation loss improvement
counter = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True) as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)
    
    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            val_loss += criterion(outputs, labels).item()
    
    # Compute average validation loss and accuracy
    val_loss /= len(val_loader)
    accuracy = 100. * correct / total
    
    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0  # Reset counter
        # Save the model's state
        torch.save(model.state_dict(), r"C:\Users\aliof\models\mobilenetv3small\best_model_paper.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f'Validation loss did not improve for {patience} epochs. Early stopping...')
            break
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

C:\Users\aliof\anacondaLocation\envs\tourchgpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/50: 100%|███████████████████████████████████████| 375/375 [05:18<00:00,  1.18it/s, accuracy=90.9, loss=0.00275]


Epoch [1/50], Validation Loss: 0.0792, Validation Accuracy: 97.66%


Epoch 2/50: 100%|██████████████████████████████████████| 375/375 [02:13<00:00,  2.81it/s, accuracy=98.5, loss=0.000384]


Epoch [2/50], Validation Loss: 0.0466, Validation Accuracy: 98.69%


Epoch 3/50: 100%|██████████████████████████████████████| 375/375 [02:13<00:00,  2.81it/s, accuracy=99.3, loss=0.000193]


...

Epoch [4/50], Validation Loss: 0.0431, Validation Accuracy: 99.05%


Epoch 5/50: 100%|███████████████████████████████████████| 375/375 [02:13<00:00,  2.80it/s, accuracy=99.7, loss=7.26e-5]


Epoch [5/50], Validation Loss: 0.0486, Validation Accuracy: 98.97%


Epoch 6/50:   7%|██▉                                     | 28/375 [00:10<02:09,  2.68it/s, accuracy=99.8, loss=4.73e-5]


KeyboardInterrupt: 

In [23]:
import torch
from thop import profile
from torchvision.models import mobilenet_v3_small

# Load the pre-trained MobileNetV3 model
model = mobilenet_v3_small(pretrained=False)

# Modify the model's top layers for your specific task
num_classes = 12  # Number of classes in your dataset
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, num_classes)

#load our trained model
model.load_state_dict(torch.load(r"C:\Users\aliof\models\mobilenetv3small\best_model_paper.pth"))

# Define input tensor shape (batch_size, channels, height, width)
input_shape = (1, 3, 224, 224)  # Assuming input images are RGB with size 224x224

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate random input data for profiling
inputs = torch.randn(input_shape).to(device)

# Calculate FLOPS
flops, params = profile(model, inputs=(inputs,), verbose=False)

print(f"FLOPS: {flops / (10**6):.2f} million")
print(f"Number of parameters: {params / (10**6):.2f} million")


FLOPS: 61.47 million
Number of parameters: 1.53 million


In [2]:
train_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\test\New folder", transform=transform)
# Get class labels
class_labels = train_size.classes

# Print class labels with their index
for i, label in enumerate(class_labels):
    print(f"Index {i}: Class Label - {label}")


Index 0: Class Label - Common wheat
Index 1: Class Label - Convolvulus arvensis
Index 2: Class Label - Cotton
Index 3: Class Label - Euphorbia peplus
Index 4: Class Label - Grass
Index 5: Class Label - Lolium multiflorum
Index 6: Class Label - Maize
Index 7: Class Label - Nutgrass
Index 8: Class Label - Purslane
Index 9: Class Label - Sesame
Index 10: Class Label - Sugar beet
Index 11: Class Label - Tomato


In [4]:
#to test each class prediction accuracy
import torchvision
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert images to tensors
])

class_labels = train_size.classes

# Load dataset
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\test\New folder", transform=transform)

# Create data loader for validation set
val_loader = DataLoader(val_size, batch_size=32, shuffle=False)

# Load pre-trained MobileNetV3 model
model = torchvision.models.mobilenet_v3_small(pretrained=False)

# Modify the model's top layers for your specific task
num_classes = len(val_size.classes)  # Number of classes in your dataset
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, num_classes)

# Load the best model weights
model.load_state_dict(torch.load(r"C:\Users\aliof\models\mobilenetv3small\best_model_paper.pth"))

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Evaluate the model on the validation dataset
model.eval()
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Calculate accuracy for each class
class_accuracy = [class_correct[i] / class_total[i] if class_total[i] != 0 else 0 for i in range(num_classes)]
for i in range(num_classes):
    print(f"Accuracy for class {class_labels[i]}: {class_accuracy[i] * 100:.2f}%")


C:\Users\aliof\anacondaLocation\envs\tourchgpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aliof\anacondaLocation\envs\tourchgpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.73it/s]

Accuracy for class Common wheat: 90.91%
Accuracy for class Convolvulus arvensis: 100.00%
Accuracy for class Cotton: 25.00%
Accuracy for class Euphorbia peplus: 7.14%
Accuracy for class Grass: 100.00%
Accuracy for class Lolium multiflorum: 100.00%
Accuracy for class Maize: 100.00%
Accuracy for class Nutgrass: 100.00%
Accuracy for class Purslane: 100.00%
Accuracy for class Sesame: 100.00%
Accuracy for class Sugar beet: 75.00%
Accuracy for class Tomato: 42.86%


In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define lists to store metrics
class_precision = []
class_recall = []
class_f1_score = []

# Evaluate the model on the validation dataset
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate precision, recall, and F1-score for each class
for i in range(num_classes):
    precision = precision_score(true_labels, predictions, labels=[i], average='micro')
    recall = recall_score(true_labels, predictions, labels=[i], average='micro')
    f1 = f1_score(true_labels, predictions, labels=[i], average='micro')
    class_precision.append(precision)
    class_recall.append(recall)
    class_f1_score.append(f1)
    print(f"Metrics for class {class_labels[i]}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1-score: {f1:.2f}")


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 10.02it/s]

Metrics for class Common wheat:
  Precision: 1.00
  Recall: 0.91
  F1-score: 0.95
Metrics for class Convolvulus arvensis:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Cotton:
  Precision: 0.25
  Recall: 0.25
  F1-score: 0.25
Metrics for class Euphorbia peplus:
  Precision: 1.00
  Recall: 0.07
  F1-score: 0.13
Metrics for class Grass:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Lolium multiflorum:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Maize:
  Precision: 0.57
  Recall: 1.00
  F1-score: 0.73
Metrics for class Nutgrass:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Purslane:
  Precision: 0.25
  Recall: 1.00
  F1-score: 0.40
Metrics for class Sesame:
  Precision: 0.88
  Recall: 1.00
  F1-score: 0.93
Metrics for class Sugar beet:
  Precision: 0.60
  Recall: 0.75
  F1-score: 0.67
Metrics for class Tomato:
  Precision: 0.38
  Recall: 0.43
  F1-score: 0.40


In [31]:
import os
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the trained model (MobileNetV3)
model = models.mobilenet_v3_small(pretrained=False)

# Modify the model's top layers for your specific task
num_classes = 12  # Number of classes in your dataset
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, num_classes)

model.load_state_dict(torch.load(r"C:\Users\aliof\models\mobilenetv3small\best_model_paper.pth"))  # Load your trained MobileNetV3 model here
model.eval()

# Folder containing images
folder_path = r"C:\Users\aliof\OneDrive\Desktop\tomatocut"

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)

        # Apply transformations
        input_image = transform(image).unsqueeze(0)  # Add a batch dimension

        # Use GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_image = input_image.to(device)
        model = model.to(device)

        # Get predictions
        with torch.no_grad():
            output = model(input_image)
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            predicted_class = torch.argmax(probabilities).item()

        class_labels = train_size.classes

        # Print prediction for each image
        print("Image:", filename)
        print("Predicted Class:", class_labels[predicted_class])
        print("Probability:", probabilities[predicted_class].item())
        print("--------------------------------------")


Image: 0a9f2d5a-bc30-c5fd-4d6d-dc0b720e4b52.jpg
Predicted Class: Euphorbia peplus
Probability: 0.9999961853027344
--------------------------------------
Image: 167.png
Predicted Class: Maize
Probability: 0.9998841285705566
--------------------------------------
Image: Common-purslane-seedling.jpg
Predicted Class: Convolvulus arvensis
Probability: 0.22329333424568176
--------------------------------------
Image: corn-seedlings.jpg
Predicted Class: Maize
Probability: 0.9269290566444397
--------------------------------------
Image: download.jpg
Predicted Class: Sugar beet
Probability: 0.994665265083313
--------------------------------------
Image: DSC_0186.JPG
Predicted Class: Tomato
Probability: 0.9997785687446594
--------------------------------------
Image: DSC_0190.JPG
Predicted Class: Tomato
Probability: 0.9986100196838379
--------------------------------------
Image: euphorbiapeplus10.jpg
Predicted Class: Tomato
Probability: 0.6774348616600037
--------------------------------------
